# **Avaliação 03 - ENGA74 2023.1**
**Aluno:** André Paiva Conrado Rodrigues

## **Importação de dependências para os códigos**

In [23]:
from time import time
import numpy as np
from numpy.random import rand
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## **Funções de ***benchmark*****

Esfera:

In [24]:
#Função Esfera
def esfera(x_array, derivada=False):
    if(not derivada):
        return np.sum(np.power(x_array, 2))
    else:
        return 2 * x_array

Ackley:

In [25]:
#Função Ackley
def ackley(x_array, derivada=False):
    eps_stability = 1e-8
    exp1 = np.exp(-0.2 * np.sqrt(np.sum(np.power(x_array, 2))/x_array.shape[0]))
    exp2 = np.exp(np.sum(np.cos(2 * np.pi * x_array))/x_array.shape[0])
    
    if(not derivada):
        return (-20 * exp1) - exp2 + 20 + np.e
    else:
        aux1 = np.tile(np.sqrt(np.sum(np.power(x_array, 2)) + eps_stability), (x_array.shape[0], 1)).transpose()
        aux2 = np.divide(x_array, aux1)
        exp1 = np.tile(exp1, (x_array.shape[0], 1)).transpose()
        exp2 = np.tile(exp2, (x_array.shape[0], 1)).transpose()
        coef1 = 2.828
        coef2 = np.pi
        if(x_array.shape[0] == 3):         
            coef1 = 5.6562/np.sqrt(3)
            coef2 = 2 * np.pi / 3
        return (coef1 * aux2 * exp1) + (coef2 * exp2 * np.sin(2 * np.pi * x_array))

## **Funções de treinamento**

Gradiente Descendente:

In [26]:
def gradiente(x_array, funcao, alpha, iteracoes=30):
    x_trained_array = np.copy(x_array)
    func_evolution = [funcao(x_array)]
    time_init = time()
    for _ in range(iteracoes):
        deriv = funcao(x_trained_array, derivada=True)
        x_trained_array = x_trained_array - (alpha * deriv)
        func_evolution.append(funcao(x_trained_array))
    func_evolution = np.array([func_evolution])
    elapsed_time = time() - time_init
    return func_evolution, elapsed_time

In [27]:
def treina_gradiente(alpha, dim, funcao='esfera', iteracoes=30):
    init = (np.random.rand(20, dim) * 60) - 30

    evol_train = np.empty((0, iteracoes + 1))
    times = np.array([])

    if(funcao == 'ackley'):
        f = ackley
    else:
        f = esfera

    for i in range(20):
        func_evolution, elapsed_time = gradiente(init[i], f, alpha, iteracoes)
        evol_train = np.append(evol_train, func_evolution, axis=0)
        times = np.append(times, elapsed_time)

    evol_train=pd.DataFrame(np.transpose(evol_train))
    mean_training_time = np.mean(times) * 1000

    fig = px.line(evol_train, title=f"Treino da func. {funcao} em R{dim} com alpha {alpha} - Tempo médio para {iteracoes} iterações: {mean_training_time:.3f} ms")
    fig.update_xaxes(title_text='Iterações')
    fig.update_yaxes(title_text=f'Valor da função {funcao}')
    fig.update_layout(legend_title_text='Inicialização')
    fig.show()

Algoritmo Genético:

In [28]:
def genetico(x_matrix, funcao, iteracoes=30, p_recomb=0.1, p_mutacao = 0.05):
    x_pop = np.copy(x_matrix)
    x_fitness = np.array([-funcao(candidato) for candidato in x_pop])
    return x_fitness

    

In [64]:
df = pd.DataFrame(columns=['x', 'y', 'fitness', 'iter'])

for i in range(3):
    init = (np.random.rand(20, 2) * 60) - 30
    fit = genetico(init, esfera).reshape(-1,1)
    iter = np.tile([i], 20).reshape(-1, 1)
    init = np.concatenate((init, fit, iter), axis=1)
    df = pd.concat([df, pd.DataFrame(init, columns=df.columns)]).reset_index(drop=True)

fig = px.scatter(df, x='x', y='y', color='fitness', animation_frame='iter',
                 range_y=[-30, 30], range_x=[-30, 30])
fig.show()

Enxame de Partículas:

## **Exercício 01 - Gradiente**

### 1. Otimização da esfera em $\mathbb{R}^{2}$ com 20 inicializações:

Caso 1.1: $\alpha$ = 0.1, treinamento por 30 iterações

In [30]:
treina_gradiente(1e-1, 2, 'esfera', 30)

Caso 1.2: $\alpha$ = 0.01, treinamento por 30 iterações

In [31]:
treina_gradiente(1e-2, 2, 'esfera', 30)

Caso 1.3: $\alpha$ = 10, treinamento por 30 iterações

In [32]:
treina_gradiente(1e1, 2, 'esfera', 30)

### 2. Otimização da esfera em $\mathbb{R}^{3}$ com 20 inicializações:

Caso 2.1: $\alpha$ = 0.1, treinamento por 30 iterações

In [33]:
treina_gradiente(1e-1, 3, 'esfera', 30)

Caso 2.2: $\alpha$ = 0.01, treinamento por 30 iterações

In [34]:
treina_gradiente(1e-2, 3, 'esfera', 30)

Caso 2.3: $\alpha$ = 10, treinamento por 30 iterações

In [35]:
treina_gradiente(1e1, 3, 'esfera', 30)

### 3. Otimização da função Ackley em $\mathbb{R}^{2}$ com 20 inicializações:

Caso 3.1: $\alpha$ = 0.01, treinamento por 30 iterações

In [36]:
treina_gradiente(1e-2, 2, 'ackley', 30)

Caso 3.2: $\alpha$ = 0.1, treinamento por 30 iterações

In [37]:
treina_gradiente(1e-1, 2, 'ackley', 30)

### 4. Otimização da função Ackley em $\mathbb{R}^{3}$ com 20 inicializações:

Caso 4.1: $\alpha$ = 0.005, treinamento por 30 iterações

In [38]:
treina_gradiente(5e-3, 3, 'ackley', 30)

Caso 4.2: $\alpha$ = 0.01, treinamento por 30 iterações

In [39]:
treina_gradiente(1e-2, 3, 'ackley', 30)

## **Exercício 02 - Algoritmo Genético**

## **Exercício 03 - Enxame de Partículas**

## **Exercício 04 - Considerações**